In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt

import torch

import pyciemss
from pyciemss.PetriNetODE.interfaces import (setup_petri_model, reset_model, intervene_petri_model,
                                             sample_petri, calibrate_petri, optimize, load_petri_model)
import mira
from pyciemss.utils import get_tspan
from pyciemss.utils.interface_utils import convert_to_output_format
from pyciemss.visuals import plots
import pandas as pd

import warnings
warnings.filterwarnings('ignore')


# Deterministic Lotka Volterra

## Load model

In [3]:
from pyciemss.PetriNetODE.models import LotkaVolterra

raw_deterministic_lotka_volterra = LotkaVolterra(alpha=.67, beta=1.33, gamma=1.00, delta=1.00, add_uncertainty=False)
raw_deterministic_lotka_volterra

LotkaVolterra()

## Initialize model

In [4]:
deterministic_lotka_volterra = setup_petri_model(raw_deterministic_lotka_volterra, start_time=0.0, start_state=dict(prey_population=1.0, predator_population=1.0))
deterministic_lotka_volterra

LotkaVolterra()

In [5]:
tspan = get_tspan(1, 50, 500)

## Sample trajectory

In [6]:
deterministic_lotka_volterra_trajectory = sample_petri(deterministic_lotka_volterra, tspan, 1)
deterministic_lotka_volterra_trajectory['prey_population_sol'][0][0]

tensor(0.5775)

In [17]:
deterministic_lotka_volterra_trajectory_df = convert_to_output_format(deterministic_lotka_volterra_trajectory, tspan, nice=True)
schema = plots.trajectories(deterministic_lotka_volterra_trajectory_df)
plots.ipy_display(schema)


# Load lotka volterra  with uncertainty

In [8]:
from pyciemss.PetriNetODE.models import LotkaVolterra

uncertain_lotka_volterra_raw= LotkaVolterra(alpha=0.67, beta=1.33, gamma=1.0, delta=1.0, pseudocount=10)
uncertain_lotka_volterra_raw

LotkaVolterra()

## Initialize uncertain Lotka-volterra model

In [9]:
uncertain_lotka_volterra = setup_petri_model(uncertain_lotka_volterra_raw,
                                             start_time=0.0, 
                                             start_state=dict(
                                                 prey_population=1.00, 
                                                 predator_population=1.00))
uncertain_lotka_volterra

LotkaVolterra()

## Sample from prior

In [10]:

prior_samples = sample_petri(uncertain_lotka_volterra, tspan, 50)
prior_samples.keys()

dict_keys(['alpha', 'beta', 'gamma', 'delta', 'prey_population_sol', 'predator_population_sol'])

In [26]:
prior_samples_df = convert_to_output_format(prior_samples, tspan, nice=True)
schema = plots.trajectories(prior_samples_df, 
                           subset=".*_sol",
                           relabel={"prey_population_sol": "prey_prior",
                                    "predator_population_sol": "predator_prior"})
plots.ipy_display(schema)

## Calibrate uncertain Lotka Volterra

We make one observation per day. Because we simulate 500 data points over 50 days, we need to sample every 10th timepoint.

In [13]:
fitted_parameters = calibrate_petri(uncertain_lotka_volterra, 
                                data =[(i, dict(prey_population=deterministic_lotka_volterra_trajectory['prey_population_sol'][0][i*10]))
                                       for i in range(1,50)])
fitted_parameters

AutoLowRankMultivariateNormal()

In [18]:
posterior_samples = sample_petri(uncertain_lotka_volterra, 
                                 timepoints=tspan, num_samples=50, 
                           inferred_parameters=fitted_parameters)

posterior_samples.keys()


dict_keys(['alpha', 'beta', 'gamma', 'delta', 'prey_population_sol', 'predator_population_sol'])

In [28]:
posterior_samples_df = convert_to_output_format(posterior_samples, tspan, nice=True)

distributions = pd.DataFrame({
    'prey_prior': prior_samples_df['prey_population_sol'], 
    'predator_prior': prior_samples_df['predator_population_sol'],
    'prey_posterior': posterior_samples_df['prey_population_sol'],
    'predator_posterior': posterior_samples_df['predator_population_sol'],
    })

traces = pd.DataFrame({
    'prey_data': deterministic_lotka_volterra_trajectory_df['prey_population_sol'],
    'predator_data': deterministic_lotka_volterra_trajectory_df['predator_population_sol']
})

schema = plots.trajectories(distributions, traces=traces)
plots.ipy_display(schema)

## Intervene on uncertain Lotka Volterra

In [29]:
intervened_lotka_volterra = intervene_petri_model(uncertain_lotka_volterra,
                                                  [(25, 'alpha', 1.0)])
intervened_lotka_volterra

LotkaVolterra()

## Sample from intervened model

At $t=25$ we intervene to set $\alpha=1.0$.

In [30]:
intervened_samples = sample_petri(intervened_lotka_volterra,
                                  timepoints=tspan, num_samples = 50
                                  )
intervened_samples

{'alpha': tensor([0.6758, 0.6521, 0.6038, 0.7206, 0.6327, 0.7138, 0.6359, 0.6671, 0.6593,
         0.6114, 0.6595, 0.7220, 0.6242, 0.6705, 0.6613, 0.6647, 0.6043, 0.7341,
         0.6331, 0.6145, 0.6928, 0.7358, 0.7228, 0.7245, 0.7059, 0.6583, 0.6988,
         0.7329, 0.7244, 0.6386, 0.6067, 0.7197, 0.6996, 0.6723, 0.6110, 0.6166,
         0.6197, 0.6898, 0.6651, 0.6220, 0.6423, 0.6081, 0.6807, 0.6838, 0.6621,
         0.7311, 0.6302, 0.6978, 0.6300, 0.7175]),
 'beta': tensor([1.2645, 1.2464, 1.4498, 1.3406, 1.4348, 1.4094, 1.3298, 1.2329, 1.2949,
         1.2878, 1.2983, 1.4521, 1.3735, 1.3161, 1.2072, 1.3465, 1.2948, 1.3588,
         1.2036, 1.3215, 1.4146, 1.4511, 1.2347, 1.3324, 1.2498, 1.3095, 1.3459,
         1.3207, 1.2132, 1.3252, 1.4323, 1.3660, 1.3808, 1.4279, 1.4254, 1.2307,
         1.3261, 1.3584, 1.4304, 1.3485, 1.4275, 1.2321, 1.2977, 1.2651, 1.2727,
         1.2184, 1.4200, 1.3351, 1.3195, 1.3298]),
 'gamma': tensor([1.0891, 1.0745, 0.9349, 0.9072, 0.9113, 0.9462, 1.025

In [32]:
intervened_samples_df = convert_to_output_format(intervened_samples, tspan, nice=True)
distributions = pd.DataFrame({
    'prey_prior': prior_samples_df['prey_population_sol'], 
    'predator_prior': prior_samples_df['predator_population_sol'],
    'prey_intervened': intervened_samples_df['prey_population_sol'],
    'predator_intervened': intervened_samples_df['predator_population_sol'],
    })

schema = plots.trajectories(distributions,
                           markers={"Intervention at T=25": 25})
schema = plots.resize(schema, w=700, h=400)
plots.ipy_display(schema)

## Sample from counterfactual distribution

Given that we observe the deterministic trajectory in the factual world,
what would be the trajectory in a counterfactual world where we intervene at $t=25$ to set $\alpha=1.5$?

In [33]:
counterfactual_samples = sample_petri(intervened_lotka_volterra,
                                  timepoints=tspan, num_samples = 50,
                                      inferred_parameters=fitted_parameters
                                  )
counterfactual_samples

{'alpha': tensor([0.6626, 0.6615, 0.6613, 0.6643, 0.6640, 0.6629, 0.6670, 0.6672, 0.6615,
         0.6590, 0.6590, 0.6674, 0.6640, 0.6621, 0.6597, 0.6675, 0.6602, 0.6668,
         0.6616, 0.6684, 0.6641, 0.6681, 0.6542, 0.6654, 0.6635, 0.6623, 0.6638,
         0.6628, 0.6682, 0.6633, 0.6587, 0.6682, 0.6679, 0.6698, 0.6568, 0.6599,
         0.6643, 0.6658, 0.6650, 0.6725, 0.6656, 0.6652, 0.6561, 0.6661, 0.6752,
         0.6669, 0.6668, 0.6615, 0.6635, 0.6596]),
 'beta': tensor([1.2940, 1.2218, 1.2384, 1.2306, 1.2352, 1.2445, 1.2431, 1.2460, 1.2154,
         1.2632, 1.2548, 1.2433, 1.2509, 1.2339, 1.2378, 1.2408, 1.2347, 1.2388,
         1.2309, 1.2672, 1.2574, 1.2403, 1.2763, 1.2862, 1.2570, 1.2360, 1.2893,
         1.2435, 1.2435, 1.2157, 1.2567, 1.3116, 1.2704, 1.2425, 1.2409, 1.2573,
         1.2497, 1.2258, 1.2343, 1.2424, 1.2317, 1.2444, 1.2375, 1.2451, 1.2680,
         1.3091, 1.2558, 1.2631, 1.2220, 1.2323]),
 'gamma': tensor([1.0173, 1.0035, 1.0139, 1.0046, 1.0133, 1.0124, 1.010

In [34]:
counterfactual_samples_df = convert_to_output_format(counterfactual_samples, tspan, nice=True)
dists = pd.DataFrame(
    {'prey_posterior': posterior_samples_df['prey_population_sol'], 
     'predator_posterior': posterior_samples_df['predator_population_sol'],
     'prey_counterfactual': counterfactual_samples_df['prey_population_sol'],
     'predator_counterfactual': counterfactual_samples_df['predator_population_sol']
    })

schema = plots.trajectories(dists, markers={"Intervention at T=25": 25})
plots.ipy_display(schema)